In [98]:
import psycopg2
import json
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Константы

### Дата начала и окончания периода для расчета

In [99]:
start_quarter = datetime.datetime(2023, 1, 1) # включительно
end_quarter = datetime.datetime(2023, 4, 1)   # НЕ включительно

### Ид конфигов анкет

In [100]:
# Ид конфига анкеты опроса соискателей
anket_soisk_config_id = 167679

# Ид конфига анкеты опроса работодателей
anket_rab_config_id = 167682

### Параметры подключения к БД

In [101]:
db_host = ""
db_port=""
db_database=""
db_user=""
db_password=""

### Сопоставление вопросов анкет и компонентов индексов

In [102]:
questions_dict = {
    'Dr1': 'q10',
    'Dr2': 'q11',
    'Dr3': 'q12.1',
    'Pr1': 'q1',
    'Mr1': 'q15',
    'Mr2': 'q16',
    'Mr3': 'q17',
    'Mr4': 'q18',
    'Mr6': 'q9',
    'Mr7': 'q8',
    'Mr8': 'q13.2',
    'Ar1': 'q4',
    'Ar2': 'q5',
    'Kr1': 'q7',
    'Kr2': 'q6',
    'Kr3': 'q3',
    'Kr4': 'q2'
}

# добавляем точно такие же пары индекс-вопрос, но для соискателей (то есть Ds1: q10, Ds2: q11 и тод)
resQuestionsDict = {}
for q in questions_dict:
    resQuestionsDict[q] = questions_dict[q]
    resQuestionsDict[q.replace('r', 's')] = questions_dict[q]
questions_dict = resQuestionsDict

# Получаем исходные данные из БД

In [103]:
# Подключаемся к БД
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    database=db_database,
    user=db_user,
    password=db_password)
cur = conn.cursor()

### Список регионов

In [104]:
cur.execute('select * from region')
regions_fetched = cur.fetchall()

### Список ЦЗН

In [105]:
cur.execute('select * from abstract_department')
czns_fetched = cur.fetchall()

### Анкеты соискателей

In [106]:
cur.execute('SELECT * from anket_form_data where config_id=' + str(anket_soisk_config_id))
ankets_soisk_fetched = cur.fetchall()

### Анкеты работодателей

In [107]:
cur.execute('SELECT * from anket_form_data where config_id=' + str(anket_rab_config_id))
ankets_rab_fetched = cur.fetchall()

# Предобработка исходных данных

### Список регионов

In [108]:
# выбираем только нужные колонки и правильно их называем
regions_df = pd.DataFrame(regions_fetched)
regions_df = regions_df[[0, 1, 3]].rename(columns={0: 'region_code', 1: 'region_name', 3: 'net_type'})
regions_df.head(5) # покажем первые несколько записей

,region_code,region_name,net_type
0,VLADIM,Владимирская область,CENTRALIZED
1,BURYAT,Республика Бурятия,DECENTRALIZED
2,BASHK,Республика Башкортостан,DECENTRALIZED
3,NIZHEG,Нижегородская область,CENTRALIZED
4,YAROSL,Ярославская область,DECENTRALIZED


### Список ЦЗН

In [109]:
# выбираем только нужные колонки и правильно их называем
czns_df = pd.DataFrame(czns_fetched)[[0,1,3]]
czns_df = czns_df.rename(columns={0: 'type', 1: 'cznId', 3: 'region_code'})
# присоединяем информацию о регионе
czns_df = czns_df.merge(regions_df, on='region_code')
czns_df.head(5) # покажем первые несколько записей

,type,cznId,region_code,region_name,net_type
0,territorial_czn,167231,NIZHEG,Нижегородская область,CENTRALIZED
1,territorial_czn,167493,NIZHEG,Нижегородская область,CENTRALIZED
2,territorial_czn,167138,NIZHEG,Нижегородская область,CENTRALIZED
3,territorial_czn,167491,NIZHEG,Нижегородская область,CENTRALIZED
4,territorial_czn,179081,NIZHEG,Нижегородская область,CENTRALIZED


### Анкеты соискателей

In [110]:
ankets_soisk_fetched[0]
# пример записи о заполненной анкете
# по порядку: 0) id анкеты, 1) json-строка с ответами, 2) нерелевантное поле, 
# 3) ид конфига, 4) ид цзн, 5) нерелевантное поле, 6) дата и время заполнения,
# 7) 8) нерелевантное поле 9) браузер заполнившего 10) ip заполнившего 11) нерелевантное поле

(782778,
 '[[9,9,9,0,9,9,9,9,9,9,9,0,9,1,0,9,8,9,7],[1,2,9]]',
 None,
 167679,
 '178028',
 None,
 datetime.datetime(2023, 3, 9, 15, 33, 18, 949486),
 None,
 None,
 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Mobile/15E148 Safari/604.1',
 '127.0.0.1',
 None)

In [111]:
# ответы на анкету представляются строкой вида [[9,9,9,0,9,9,9,9,9,9,9,0,9,1,0,9,8,9,7],[1,2,9]]
# нужно уметь представлять ответы в виде питонячего массива
# здесь первый подмассив - шаг 1, второй - шаг 2
# но есть нюанс. Есть вопросы 12.1 13.1 13.2, которые появляются только при утвердительном (0) ответе на вопросы 12 и 13
def parse_soisk_answers(answers_str):
    i = 11
    a = json.loads(answers_str)[0]
    if a[i] == 1:
        a.insert(i+1, None)
    i = i+2
    if a[i] == 1:
        a.insert(i+1, None)
        a.insert(i+2, None)
    return a


In [112]:
# датасет с анкетами будет содержать следующие колонки
headers = ['id', 'configId', 'cznId', 'date', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q12.1', 'q13', 'q13.1', 'q13.2', 'q14', 'q15', 'q16', 'q17', 'q18']
# формируем датасет по анкетам с разворачиванием ответов в колнки
ankets_soisk_df = pd.DataFrame([[it[0], it[3], int(it[4]), it[6]] + (parse_soisk_answers(it[1])) for it in ankets_soisk_fetched], columns=headers)


In [113]:
# Посмотрим несколько строк из датасета с анкетами работодателей
ankets_soisk_df.head(3)

,id,configId,cznId,date,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q12.1,q13,q13.1,q13.2,q14,q15,q16,q17,q18
0,782778,167679,178028,2023-03-09 15:33:18.949486,9,9,9,0,9,9,9,9,9,9,9,0,9.0,1,None,NaN,0,9,8,9,7
1,783391,167679,167102,2023-03-09 17:58:49.598947,9,5,9,0,9,9,9,9,9,7,7,1,NaN,1,None,NaN,1,6,7,6,9
2,783393,167679,167102,2023-03-09 17:59:39.923987,9,9,9,0,9,9,9,9,9,9,9,1,NaN,1,None,NaN,0,9,9,9,9


In [114]:
# Сколько всего заполнено анкет соискателей по подходящим датам
len(ankets_soisk_df)

5787

### Анкеты работодателей

In [115]:
# парсинг ответов на анкету работодателей на текущий момент такой же, как для соискателей
parse_rab_answers = parse_soisk_answers

In [116]:
# датасет с анкетами будет содержать следующие колонки
headers = ['id', 'configId', 'cznId', 'date', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q12.1', 'q13', 'q13.1', 'q13.2', 'q14', 'q15', 'q16', 'q17', 'q18']
# формируем датасет по анкетам с разворачиванием ответов в колнки
ankets_rab_df = pd.DataFrame([[it[0], it[3], int(it[4]), it[6]] + (parse_rab_answers(it[1])) for it in ankets_rab_fetched], columns=headers)


In [117]:
# отбираем только анкеты, которые находятся в границах start_quarter и end_quarter
ankets_rab_df = ankets_rab_df[ankets_rab_df['date'] >= start_quarter]
ankets_rab_df = ankets_rab_df[ankets_rab_df['date'] < end_quarter]

In [118]:
# Посмотрим несколько строк из датасета с анкетами работодателей
ankets_rab_df.head(3)

,id,configId,cznId,date,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q12.1,q13,q13.1,q13.2,q14,q15,q16,q17,q18
0,781943,167682,179085,2023-03-09 13:15:30.581698,6,9,9,0,9,9,7,4,9,9,9,0,8.0,1,None,NaN,1,7,5,1,1
1,784961,167682,167231,2023-03-10 11:01:30.637920,9,9,9,0,9,9,9,9,9,9,9,0,9.0,1,None,NaN,0,9,9,9,9
2,785557,167682,179085,2023-03-10 12:37:52.988919,1,4,4,1,1,2,3,2,0,1,1,1,NaN,1,None,NaN,1,4,4,4,4


In [119]:
# Сколько всего заполнено анкет работдателей по подходящим датам
len(ankets_rab_df)

1372

# Расчет компонентов индекса по чеклистам

In [120]:
czns_df_with_checklist_answers = czns_df.copy()

In [121]:
# На текущий момент заполненных чеклистов почти нет. Без чеклистов индексы считаться не должны, но ставим временно все компоненты в 0
czns_df_with_checklist_answers['R'] = 0
czns_df_with_checklist_answers['DO'] = 0
czns_df_with_checklist_answers['DS'] = 0
czns_df_with_checklist_answers['DSS'] = 0
czns_df_with_checklist_answers['DT'] = 0
czns_df_with_checklist_answers['Ps2'] = 0
czns_df_with_checklist_answers['Ps3'] = 0
czns_df_with_checklist_answers['Pr2'] = 0
czns_df_with_checklist_answers['Pr3'] = 0
czns_df_with_checklist_answers['Ms5'] = 0
czns_df_with_checklist_answers['Mr5'] = 0
czns_df_with_checklist_answers['SK'] = 0
czns_df_with_checklist_answers['PZ'] = 0
czns_df_with_checklist_answers['VV'] = 0
czns_df_with_checklist_answers['TrD'] = 0
czns_df_with_checklist_answers['KP'] = 0

# Расчет компонентов индекса по анкетам

### Вспомогательные функции

In [122]:
# считает долю ответов >=8 в общем числе ответв
# df - датафрейм из одной колонки
def good_ratio(df):
    # df>=7, а не 8, потому что ответы по шкале от 1 до 10 представлены числами от 0 до 9
    return df[df>=7].count() / df.count() if df.count() != 0 else None

# добавляет в датасет колонку с долей ответов >= 8
# input_czns_df - куда добавить колонку
# input_ankets_dfq - датасет с ответами
# index_component - компонент индекса по справочнику
def addGoodRatioQ(input_czns_df, input_ankets_dfq, index_component):
    ankets_dfq_good_ratio = input_ankets_dfq.groupby('cznId')[questions_dict[index_component]].agg(good_ratio)
    return input_czns_df.join(ankets_dfq_good_ratio, on='cznId').rename(columns={questions_dict[index_component]: index_component})

In [123]:
# Из варианта да-нет-не знаю дает долю тех, кто да из тех, кто знает
def yes_ratio(df):
    threshold = 0.8 # порог, ниже которого будет 0
    yes_count = df[df==0].count()
    no_count = df[df==1].count()
    if yes_count + no_count == 0:
        return None
    ratio = yes_count / (yes_count + no_count)
    return ratio if ratio >= threshold else 0

# добавляет в датасет колонку с долей ответов "да" (см yes_ratio выше)
# input_czns_df - куда добавить колонку
# input_ankets_dfq - датасет с ответами
# index_component - компонент индекса по справочнику
def addYesRatioQ(input_czns_df, input_ankets_dfq, index_component):
    ankets_dfq_good_ratio = input_ankets_dfq.groupby('cznId')[questions_dict[index_component]].agg(yes_ratio)
    return input_czns_df.join(ankets_dfq_good_ratio, on='cznId').rename(columns={questions_dict[index_component]: index_component})

### Добавим количество анкет у ЦЗН и уберем ЦЗН без анкет

#### Для соискателей

In [124]:
# считаем количество анкет соискателей, заполненных для каждого ЦЗН
czn_soisk_ankets_count = ankets_soisk_df.groupby('cznId')['id'].count()
czns_with_soisk_ankets = czns_df.join(czn_soisk_ankets_count, on='cznId').rename(columns={'id': 'soisk_ankets_count'})
# оставляем только ЦЗН, где есть заполненные анкеты соискателей
czns_with_soisk_ankets = czns_with_soisk_ankets[czns_with_soisk_ankets['soisk_ankets_count']>0]


In [125]:
# добавим для отобранных ЦЗН значения компонентов индекса по чеклистам
czns_with_soisk_ankets = czns_with_soisk_ankets.merge(czns_df_with_checklist_answers.drop(columns=['type', 'region_code', 'region_name', 'net_type']), on='cznId')
czns_with_soisk_ankets.head(3) # покажем несколько строк таблицы ЦЗН с анкетами


,type,cznId,region_code,region_name,net_type,soisk_ankets_count,R,DO,DS,DSS,DT,Ps2,Ps3,Pr2,Pr3,Ms5,Mr5,SK,PZ,VV,TrD,KP
0,territorial_czn,167231,NIZHEG,Нижегородская область,CENTRALIZED,90.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,territorial_czn,179085,NIZHEG,Нижегородская область,CENTRALIZED,155.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,territorial_czn,176596,BASHK,Республика Башкортостан,DECENTRALIZED,380.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Для работодателей

In [126]:
# считаем количество анкет работодателей, заполненных для каждого ЦЗН
czn_rab_ankets_count = ankets_rab_df.groupby('cznId')['id'].count()
czns_with_rab_ankets = czns_df.join(czn_rab_ankets_count, on='cznId').rename(columns={'id': 'rab_ankets_count'})
# оставляем только ЦЗН, где есть заполненные анкеты работодателей
czns_with_rab_ankets = czns_with_rab_ankets[czns_with_rab_ankets['rab_ankets_count']>0]

In [127]:
# добавим для отобранных ЦЗН значения компонентов индекса по чеклистам
czns_with_rab_ankets = czns_with_rab_ankets.merge(czns_df_with_checklist_answers.drop(columns=['type', 'region_code', 'region_name', 'net_type']), on='cznId')

### Каждый компонент индекса будем добавлять вспомогательной функцией

In [128]:
# df - датафрейм, куда добавляем
# ankets_dfq - датафрейм с ответами
# index_group - 'D', 'M', 'A', 'K' - группа компонента индекса
# anket_type - 's' или 'r' (соискатели и работодатели соответственно)
# component_num - номер индекса.
# из трех последних параметров получаем Ds4 и тп
# formula_type - good_ratio или yes_ratio (доля ответв >= 8 или доля ответв "да")
# coef - коэффициент-множитель полученного балла
# add_checklist_component - какой компонент индекса чеклиста добавить (R, DO, DS, DSS и так далее)
def add_index_component(df, ankets_dfq, index_group, anket_type, component_num, formula_type, coef, add_checklist_component):
    index_component = index_group+anket_type+str(component_num) # Например, 'M' + 'r' + 2 - субиндекс Мr2
    
    if anket_type != 's' and anket_type != 'r':
         raise ValueError('unknown anket_type: ' + anket_type + '. Should be s or r')

    if formula_type == 'good_ratio':
        df = addGoodRatioQ(df, ankets_dfq, index_component)
    elif formula_type == 'yes_ratio':
        df = addYesRatioQ(df, ankets_dfq, index_component)
    else:
        raise ValueError('unknown formula_type: ' + formula_type + '. Should be good_ratio or yes_ratio')
    
    df[index_component] = df[index_component] * coef
    if(add_checklist_component):
        df[index_component] = df[index_component] + df[add_checklist_component]
    return df

### Опишем компоненты индексов из анкет и формулы расчета

In [129]:
# 1й элемент: 'D', 'M', 'A', 'K' - группа компонента индекса
# 2й элемент: номер компонента индекса
# 3й элемент: 'good_ratio' или 'yes_ratio' - доля ответов >=8 или доля ответов "да"
# 4й элемент: 3 - коэффициент, на который нужно умножить эту долю
# какой компонент индекса чеклиста добавить (R, DO, DS, DSS и так далее). None, если добавлять не нужно
index_anket_components = [
    ('D', 1, 'good_ratio', 3, None),
    ('D', 2, 'good_ratio', 3, None),
    ('D', 3, 'good_ratio', 1, 'R'),
    ('P', 1, 'good_ratio', 2, None),
    ('M', 1, 'good_ratio', 1, 'DO'),
    ('M', 2, 'good_ratio', 1, 'DS'),
    ('M', 3, 'good_ratio', 1, 'DSS'),
    ('M', 4, 'good_ratio', 1, 'DT'),
    ('M', 6, 'good_ratio', 1, 'SK'),
    ('M', 7, 'good_ratio', 3, None),
    ('M', 8, 'good_ratio', 1, 'PZ'),
    ('A', 1, 'yes_ratio', 2, None),
    ('A', 2, 'good_ratio', 2, None),
    ('K', 1, 'good_ratio', 1, 'VV'),
    ('K', 2, 'good_ratio', 1, None),
    ('K', 3, 'good_ratio', 1, 'TrD'),
    ('K', 4, 'good_ratio', 1, 'KP'),
]

### Основная функция расчета

In [130]:
# df - датафрейм, куда добавляем индексы
# ankets_dfq - датафрейм с ответами
def compute_anket_index_components(df, ankets_dfq, anket_type):
    for iac in index_anket_components:
        df = add_index_component(df, ankets_dfq, iac[0], anket_type, iac[1], iac[2], iac[3], iac[4])
    return df


In [131]:
# посчитаем для соискателей
soisk_components_result = compute_anket_index_components(czns_with_soisk_ankets, ankets_soisk_df, 's')
soisk_components_result.head(3) # покажем несколько результатов

,type,cznId,region_code,region_name,net_type,soisk_ankets_count,R,DO,DS,DSS,DT,Ps2,Ps3,Pr2,Pr3,Ms5,Mr5,SK,PZ,VV,TrD,KP,Ds1,Ds2,Ds3,Ps1,Ms1,Ms2,Ms3,Ms4,Ms6,Ms7,Ms8,As1,As2,Ks1,Ks2,Ks3,Ks4
0,territorial_czn,167231,NIZHEG,Нижегородская область,CENTRALIZED,90.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.833333,2.766667,0.923077,1.866667,0.922222,0.788889,0.888889,0.733333,0.955556,2.733333,1.000000,0.00000,1.866667,0.944444,0.922222,0.822222,0.911111
1,territorial_czn,179085,NIZHEG,Нижегородская область,CENTRALIZED,155.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.361290,2.380645,0.583333,1.690323,0.748387,0.606452,0.625806,0.516129,0.774194,2.264516,0.727273,1.86755,1.509677,0.716129,0.793548,0.806452,0.690323
2,territorial_czn,176596,BASHK,Республика Башкортостан,DECENTRALIZED,380.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.984211,3.000000,1.000000,1.963158,0.997368,0.944737,0.936842,0.944737,0.997368,2.952632,1.000000,1.89011,1.973684,1.000000,0.992105,0.965789,0.992105


In [132]:
# посчитаем для работодателей
rab_components_result = compute_anket_index_components(czns_with_rab_ankets, ankets_rab_df, 'r')
rab_components_result.head(3) # покажем несколько результатов

,type,cznId,region_code,region_name,net_type,rab_ankets_count,R,DO,DS,DSS,DT,Ps2,Ps3,Pr2,Pr3,Ms5,Mr5,SK,PZ,VV,TrD,KP,Dr1,Dr2,Dr3,Pr1,Mr1,Mr2,Mr3,Mr4,Mr6,Mr7,Mr8,Ar1,Ar2,Kr1,Kr2,Kr3,Kr4
0,territorial_czn,167231,NIZHEG,Нижегородская область,CENTRALIZED,20.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.700000,3.000000,1.000,1.800000,1.000000,0.950000,0.900000,0.750000,0.900000,2.400000,1.0,0.000000,1.800000,0.950000,0.950000,0.950000,0.950000
1,territorial_czn,179085,NIZHEG,Нижегородская область,CENTRALIZED,79.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.012658,2.126582,0.625,0.936709,0.696203,0.594937,0.443038,0.518987,0.632911,2.050633,NaN,0.000000,1.316456,0.746835,0.670886,0.594937,0.594937
2,territorial_czn,176596,BASHK,Республика Башкортостан,DECENTRALIZED,89.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,2.966292,1.000,1.910112,1.000000,0.955056,0.898876,0.932584,0.977528,3.000000,1.0,1.953488,1.977528,1.000000,0.988764,0.898876,0.988764


# Сводные индексы

In [133]:
# df - датафрейм, куда добавляем индексы
# anket_type - 's' или 'r' (соискатели и работодатели соответственно)
def compute_sub_integral(df, anket_type):
    # Разбиваем датасет на 3 типа ЦЗН по net_type и type
    czns_mngr = df[df['type']=='manager_czn'].copy()
    czns_terr = df[df['type']=='territorial_czn']
    czns_terr_cen = czns_terr[czns_terr['net_type']=='DECENTRALIZED'].copy()
    czns_terr_dec = czns_terr[czns_terr['net_type']=='CENTRALIZED'].copy()
    
    # Считаем сводные индексы для CENTRALIZED territorial_czn
    pds = czns_terr_cen
    czns_terr_cen['D'+anket_type+'_svod'] = (pds['D'+anket_type+'1'] + pds['D'+anket_type+'2'] + pds['D'+anket_type+'3'])*2
    czns_terr_cen['P'+anket_type+'_svod'] = (pds['P'+anket_type+'1'] + pds['P'+anket_type+'2'] + pds['P'+anket_type+'3'])
    czns_terr_cen['M'+anket_type+'_svod'] = pds['M'+anket_type+'1'] + pds['M'+anket_type+'6']
    czns_terr_cen['A'+anket_type+'_svod'] = pds['A'+anket_type+'1'] + pds['A'+anket_type+'2']
    czns_terr_cen['K'+anket_type+'_svod'] = pds['K'+anket_type+'1'] + pds['K'+anket_type+'2'] + pds['K'+anket_type+'4']
    czns_terr_cen['I'+anket_type] = pds['D'+anket_type+'_svod'] + pds['P'+anket_type+'_svod'] + pds['M'+anket_type+'_svod'] + pds['A'+anket_type+'_svod'] + pds['K'+anket_type+'_svod']
    
    # Считаем сводные индексы для DECENTRALIZED territorial_czn
    pds = czns_terr_dec
    czns_terr_dec['D'+anket_type+'_svod'] = (pds['D'+anket_type+'1'] + pds['D'+anket_type+'2'] + pds['D'+anket_type+'3'])*2
    czns_terr_dec['P'+anket_type+'_svod'] = (pds['P'+anket_type+'1'] + pds['P'+anket_type+'2'] + pds['P'+anket_type+'3'])
    czns_terr_dec['M'+anket_type+'_svod'] = pds['M'+anket_type+'2']+pds['M'+anket_type+'3']+pds['M'+anket_type+'4']+pds['M'+anket_type+'5']+pds['M'+anket_type+'6']+pds['M'+anket_type+'8']
    czns_terr_dec['A'+anket_type+'_svod'] = pds['A'+anket_type+'1'] + pds['A'+anket_type+'2']
    czns_terr_dec['K'+anket_type+'_svod'] = pds['K'+anket_type+'1'] + pds['K'+anket_type+'2'] + pds['K'+anket_type+'3'] + pds['K'+anket_type+'4']
    czns_terr_dec['I'+anket_type] = pds['D'+anket_type+'_svod'] + pds['P'+anket_type+'_svod'] + pds['M'+anket_type+'_svod'] + pds['A'+anket_type+'_svod'] + pds['K'+anket_type+'_svod']
    
    # Считаем сводные индексы для manager_czn
    pds = czns_mngr
    czns_mngr['D'+anket_type+'_svod'] = None
    czns_mngr['P'+anket_type+'_svod'] = None
    czns_mngr['M'+anket_type+'_svod'] = pds['M'+anket_type+'1']+pds['M'+anket_type+'2']+pds['M'+anket_type+'3']+pds['M'+anket_type+'4']+pds['M'+anket_type+'5']+pds['M'+anket_type+'6']+pds['M'+anket_type+'7']+pds['M'+anket_type+'8']
    czns_mngr['A'+anket_type+'_svod'] = None
    czns_mngr['K'+anket_type+'_svod'] = pds['K'+anket_type+'3']
    czns_mngr['I'+anket_type] = pds['M'+anket_type+'_svod'] + pds['K'+anket_type+'_svod']
    
    return pd.concat([czns_terr_cen, czns_terr_dec, czns_mngr], axis=0)

## Сводный индекс для ЦЗН

In [134]:
# посчитаем субинтегралы для соискателей и работодателей и объединим результаты
soisk_components_result_subintegrals = compute_sub_integral(soisk_components_result, 's')
# для работдателей удалим задвоенные колонки
rab_components_result_subintegrals = compute_sub_integral(rab_components_result, 'r').drop(columns=['type', 'region_code', 'region_name', 'net_type', 'R', 'DO', 'DS', 'DSS', 'DT', 'Ps2', 'Ps3', 'Pr2', 'Pr3', 'Ms5', 'Mr5', 'SK', 'PZ', 'VV', 'TrD', 'KP'])
integrals = soisk_components_result_subintegrals.merge(rab_components_result_subintegrals, on='cznId')

#считаем сводный индекс для ЦЗН
integrals['I'] = (integrals['Is'] + integrals['Ir']) / 2

integrals

,type,cznId,region_code,region_name,net_type,soisk_ankets_count,R,DO,DS,DSS,DT,Ps2,Ps3,Pr2,Pr3,Ms5,Mr5,SK,PZ,VV,TrD,KP,Ds1,Ds2,Ds3,Ps1,Ms1,Ms2,Ms3,Ms4,Ms6,Ms7,Ms8,As1,As2,Ks1,Ks2,Ks3,Ks4,Ds_svod,Ps_svod,Ms_svod,As_svod,Ks_svod,Is,rab_ankets_count,Dr1,Dr2,Dr3,Pr1,Mr1,Mr2,Mr3,Mr4,Mr6,Mr7,Mr8,Ar1,Ar2,Kr1,Kr2,Kr3,Kr4,Dr_svod,Pr_svod,Mr_svod,Ar_svod,Kr_svod,Ir,I
0,territorial_czn,176596,BASHK,Республика Башкортостан,DECENTRALIZED,380.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.984211,3.000000,1.000000,1.963158,0.997368,0.944737,0.936842,0.944737,0.997368,2.952632,1.000000,1.890110,1.973684,1.000000,0.992105,0.965789,0.992105,13.968421,1.963158,1.994737,3.863794,2.984211,24.774320,89.0,3.000000,2.966292,1.000000,1.910112,1.000000,0.955056,0.898876,0.932584,0.977528,3.000000,1.000000,1.953488,1.977528,1.000000,0.988764,0.898876,0.988764,13.932584,1.910112,1.977528,3.931016,2.977528,24.728769,24.751545
1,territorial_czn,167104,BASHK,Республика Башкортостан,DECENTRALIZED,634.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.995268,2.995268,1.000000,1.990536,0.996845,0.992114,0.988959,0.988959,0.998423,2.985804,1.000000,1.968051,1.984227,0.998423,0.996845,0.995268,0.996845,13.981073,1.990536,1.995268,3.952278,2.992114,24.911269,71.0,2.830986,2.873239,1.000000,1.718310,0.901408,0.901408,0.845070,0.873239,0.929577,2.788732,1.000000,1.723077,1.830986,0.957746,0.929577,0.915493,0.915493,13.408451,1.718310,1.830986,3.554063,2.802817,23.314626,24.112948
2,territorial_czn,167193,BASHK,Республика Башкортостан,DECENTRALIZED,150.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.800000,2.800000,0.785714,1.760000,0.933333,0.800000,0.706667,0.720000,0.906667,2.460000,0.875000,1.777778,1.680000,0.973333,0.920000,0.873333,0.906667,12.771429,1.760000,1.840000,3.457778,2.800000,22.629206,22.0,3.000000,2.863636,1.000000,1.909091,1.000000,1.000000,0.954545,0.363636,1.000000,2.454545,1.000000,0.000000,1.909091,1.000000,0.909091,0.909091,1.000000,13.727273,1.909091,2.000000,1.909091,2.909091,22.454545,22.541876
3,territorial_czn,180622,MOS,Город Москва,DECENTRALIZED,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,3.000000,1.000000,1.500000,1.000000,0.750000,1.000000,1.000000,1.000000,2.250000,NaN,2.000000,1.500000,1.000000,1.000000,0.750000,0.750000,14.000000,1.500000,2.000000,3.500000,2.750000,23.750000,1.0,3.000000,3.000000,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,NaN,0.000000,1.000000,4.000000,3.000000,NaN,NaN
4,territorial_czn,177756,KRAYARSK,Красноярский край,DECENTRALIZED,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.887500,2.775000,0.833333,1.825000,0.937500,0.825000,0.737500,0.762500,0.950000,2.700000,1.000000,1.974359,1.725000,0.962500,0.937500,0.875000,0.925000,12.991667,1.825000,1.887500,3.699359,2.825000,23.228526,20.0,3.000000,3.000000,1.000000,2.000000,1.000000,1.000000,0.850000,0.800000,1.000000,3.000000,NaN,1.700000,2.000000,1.000000,1.000000,1.000000,1.000000,14.000000,2.000000,2.000000,3.700000,3.000000,24.700000,23.964263
5,territorial_czn,177790,KRAYARSK,Красноярский край,DECENTRALIZED,82.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,3.000000,1.000000,2.000000,0.987805,0.463415,0.475610,0.353659,1.000000,2.963415,1.000000,2.000000,1.975610,1.000000,1.000000,1.000000,1.000000,14.000000,2.000000,1.987805,3.975610,3.000000,24.963415,23.0,3.000000,3.000000,1.000000,2.000000,1.000000,0.869565,0.565217,0.565217,1.000000,2.608696,1.000000,2.000000,2.000000,1.000000,1.000000,0.782609,1.000000,14.000000,2.000000,2.000000,4.000000,3.000000,25.000000,24.981707
6,territorial_czn,177704,KRAYARSK,Красноярский край,DECENTRALIZED,81.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,3.000000,NaN,2.000000,1.000000,0.987654,0.987654,0.987654,1.000000,3.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,NaN,2.000000,2.000000,4.000000,3.000000,NaN,20.0,3.000000,3.000000,NaN,1.900000,1.000000,0.950000,1.000000,1.000000,1.000000,3.000000,NaN,2.000000,1.900000,1.000000,1.000000,1.000000,1.000000,NaN,1.900000,2.0000

## Сводный индекс для регионов

In [135]:
# для всех субъектов посчитаем среднее значение по цзн
region_integrals = pd.DataFrame(integrals[integrals['type']=='territorial_czn'].groupby('region_code')['I'].mean())

In [136]:
# для централизованных регионов нужно добавить индексы управляющих цзн
# выделим эти индексы
manager_ingetrals = pd.DataFrame(integrals[integrals['type']=='manager_czn'][['region_code', 'I']])
manager_ingetrals

,region_code,I
49,KALIN,8.536764


In [144]:
ri = region_integrals.merge(manager_ingetrals, on='region_code', how='left')
ri['I_y'] = ri['I_y'].fillna(0)
ri['I'] = ri['I_x'] + ri['I_y']
ri.drop(columns=['I_x', 'I_y'])
regions_results = regions_df.merge(ri.drop(columns=['I_x', 'I_y']), on='region_code')
regions_results

,region_code,region_name,net_type,I
0,VLADIM,Владимирская область,CENTRALIZED,24.921719
1,BASHK,Республика Башкортостан,DECENTRALIZED,23.802123
2,NIZHEG,Нижегородская область,CENTRALIZED,25.023077
3,YAROSL,Ярославская область,DECENTRALIZED,22.042756
4,TULSK,Тульская область,CENTRALIZED,NaN
5,TATAR,Республика Татарстан (Татарстан),DECENTRALIZED,NaN
6,NOVOSIB,Новосибирская область,DECENTRALIZED,NaN
7,MOS,Город Москва,DECENTRALIZED,NaN
8,KRAYARSK,Красноярский край,DECENTRALIZED,24.472985
9,ORENBUR,Оренбургская область,DECENTRALIZED,23.380735


In [141]:
# выгрузим результаты в эксель
integrals.to_excel('czns_res.xlsx')

In [145]:
regions_results.to_excel('regions_res.xlsx')